# Caching MGA results.

In [ ]:
# Use results from model.energy.

In [ ]:
import os

import pypsa

In [ ]:
n = pypsa.Network("../networks/model-energy/model-energy.nc")

In [ ]:
# Solve for cost optimality.
n.optimize(solver_name="gurobi")
# n.export_to_netcdf("test.nc")
# n = pypsa.Network("test.nc")

In [ ]:
# Import MGA functionalities
# # Hack to load all functions from ../../pypsa/optimization/mga.py
import sys

from pypsa.optimization.mga import *

sys.path.append("../../pypsa/optimization")
from types import MethodType

from mga import *

n.optimize._solve_single_direction = staticmethod(
    OptimizationAbstractMGAMixin._solve_single_direction
)

n.optimize.optimize_mga_with_cache = MethodType(
    OptimizationAbstractMGAMixin.optimize_mga_with_cache, n.optimize
)
n.optimize.optimize_mga_in_multiple_directions_cache = MethodType(
    OptimizationAbstractMGAMixin.optimize_mga_in_multiple_directions_cache, n.optimize
)

In [ ]:
directions = generate_directions_random(["wind", "solar"], 100, 0)
dimensions = {
    "wind": {"Generator": {"p_nom": {"wind": 1}}},
    "solar": {"Generator": {"p_nom": {"solar": 1}}},
}

os.makedirs("cache_test", exist_ok=True)

In [ ]:
n.optimize._solve_single_direction(
    "test.nc",
    direction=directions.iloc[0].to_dict(),
    dimensions=dimensions,
    cache_key=hash_mga(n, directions, dimensions, slack=0.02),
    cache_dir="cache_test",
    snapshots=None,
    multi_investment_periods=False,
    slack=0.02,
    model_kwargs=None,
    kwargs={"solver_name": "gurobi"},
)

In [ ]:
n.optimize.optimize_mga_in_multiple_directions_cache(
    directions=directions[:5],
    dimensions=dimensions,
    cache_dir="cache_test",
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
)

In [ ]:
n.optimize.optimize_mga_in_multiple_directions_cache(
    directions=directions[:5],
    dimensions=dimensions,
    cache_dir="cache_test",
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
)

In [ ]:
n.optimize.optimize_mga_in_multiple_directions_cache(
    directions=directions[:5],
    dimensions=dimensions,
    cache_dir=None,
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
)

In [ ]:
n.optimize.optimize_mga_in_multiple_directions(
    directions=directions[:5],
    dimensions=dimensions,
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
)